In [1]:
# Crafted by Collin Miller

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [21]:
import h5py
import numpy as np
import sys

In [9]:
resize = True

In [4]:
def get_cremi_data(file_path):
    """Returns the raw data and neuron ids.
    
    """
    
    # Get neuron ids from Cremi data.
    with h5py.File(file_path, "r") as hdf:
        print ("Initializing...")
        labels = hdf['volumes']['labels']['neuron_ids'][:]
        raw = hdf['volumes']['raw'].value
        print ("Done!")
        
    return raw, labels

In [6]:
r, l = get_cremi_data("cremi_data/train/sample_A_20160501.hdf")

Initializing...
Done!


In [10]:
def ids_to_segmentation(neuron_ids):
    """
    Given a numpy array containing the neuron ids for a CREMI neural segmentation image,
    this function returns the ground segmentation image where a segmentation boundary
    is a white pixel and everything else is a black pixel.
    Args:
        neuron_ids: A numpy array containing the neuron ids of a segmentation image.
    Returns:
        A numpy array of ground truth segmentations.
    """

    edges_right = neuron_ids[:, :-1].ravel() != neuron_ids[:, 1:].ravel()
    edges_down = neuron_ids[:-1].ravel() != neuron_ids[1:].ravel()

    edges_down = edges_down.reshape((neuron_ids.shape[0] - 1, neuron_ids.shape[1]))
    edges_right = edges_right.reshape((neuron_ids.shape[0], neuron_ids.shape[1] - 1))

    segmentations = np.zeros_like(neuron_ids)

    for index, x in np.ndenumerate(edges_down):
        if x:
            segmentations[index] = 255
            segmentations[index[0] + 1][index[1]] = 255

    for index, x in np.ndenumerate(edges_right):
        if x:
            segmentations[index] = 255
            segmentations[index[0]][index[1] + 1] = 255

    return segmentations

In [24]:
# Initialize segmentation image placeholder.
gt_segmentations = np.zeros_like(l)

# Iterate through slices and save segmentations to
# ground truth segmentations array.
print("Converting to ground truth segmentations.")
print("This may take a while...")
for index, x in enumerate(l):
    progress = index/len(l) * 100
    sys.stdout.write("\rProgress\t[%d%%]" % progress)
    sys.stdout.flush()
    gt_segmentations[index] = ids_to_segmentation(x)

Converting to ground truth segmentations.
This may take a while...
Progress	[99%]

In [19]:
ids_to_segmentation(l)

ValueError: cannot reshape array of size 193750000 into shape (124,1250)

In [8]:
if resize:
    new_size = (100, 100)

    tl_corner = (randint(0, (img.shape[0] - new_size[0])),
                 randint(0, (img.shape[1] - new_size[1])))

    resized_img = preprocessing_utils.crop_2d(img, tl_corner, new_size[0], new_size[1])
    resized_gt = preprocessing_utils.crop_2d(gt, tl_corner, new_size[0], new_size[1])

(125, 1250, 1250)